# HW5: Database - Weather Prediction


In this assignment, we will create a database to analyze historical weather data and discover the relationships between major cities.

**This file would complete the first three tasks as to construct a database with all info written into it. Instead of creating multiple databases for airports and weathers, storing all information in one then extracting the tables needed is the way I prefer. airports.db would be found as the only database created for this assignment, from which all information (lag, long, ICAO, min/max temperature, correlations, etc.) can be obtained.**

1. Find a list of the 50 most travelled airports in the US and make a table containing the relevant information, such as name, nearest city, latitude, and longitude.
2. Build another table that will hold historical weather information, such as min/max temperature, humidity, and precipitation.
3. Build a web crawler that will pull historical data from Weather Underground from 2008 until now and populate your tables accordingly.

In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import *
import requests
from io import StringIO
# A fast, extensible smart progress meter
from tqdm import tqdm

**Find a list of the 50 most travelled airports in the US and make a table containing the relevant information, such as name, nearest city, latitude, and longitude.**

To achieve this, we merge the top_airports with the ICAO_airport data, and select the columns with relevant info.

In [2]:
top = pd.read_csv('hw_5_data/top_airports.csv')
icao = pd.read_csv('hw_5_data/ICAO_airports.csv')
merged_df = pd.merge(top, icao, left_on='ICAO', right_on='ident')
top_info = merged_df[['City', 'Airport', 'ICAO', 'latitude_deg', 'longitude_deg']]

# remove database in case there's already one there
!rm top_50_airports.db

engine = create_engine('sqlite:///top_50_airports.db') # Remove the database file
top_info.to_sql('top_50_airports', engine)

In [3]:
# Show the first top 5 airports amongst top 50
top_info.head(5)

,City,Airport,ICAO,latitude_deg,longitude_deg
0,Atlanta,Hartsfield-Jackson Atlanta International Airport,KATL,33.636700,-84.428101
1,Chicago,Chicago O'Hare International Airport,KORD,41.978600,-87.904800
2,Los Angeles,Los Angeles International Airport,KLAX,33.942501,-118.407997
3,Dallas-Fort Worth,Dallas/Fort Worth International Airport,KDFW,32.896801,-97.038002
4,Denver,Denver International Airport,KDEN,39.861698,-104.672997


In [4]:
# Show the last 5 airports amongst top 50
top_info.tail(5)

,City,Airport,ICAO,latitude_deg,longitude_deg
45,San Antonio,San Antonio International Airport,KSAT,29.533701,-98.469803
46,Covington,Cincinnati/Northern Kentucky International Air...,KCVG,39.048801,-84.667801
47,Milwaukee,General Mitchell International Airport,KMKE,42.947201,-87.896599
48,Dallas,Dallas Love Field,KDAL,32.847099,-96.851799
49,Indianapolis,Indianapolis International Airport,KIND,39.717300,-86.294403


**Build another table that will hold historical weather information, such as min/max temperature, humidity, and precipitation.**

Create a new table in the database to hold all the historical weather data

In [5]:
metadata = MetaData()
weather_table = Table('weather', metadata,
                      Column('icao', String),
                      Column('date', String),
                      Column('min_temp', String),
                      Column('max_temp', String),
                      Column('mean_temp', String),
                      Column('min_humidity', String),
                      Column('max_humidity', String),
                      Column('mean_humidity', String),
                      Column('precip', String),
                      Column('event', String))
weather_table.create(bind=engine)

**Build a web crawler that will pull historical data from Weather Underground from 2008 until now and populate your tables accordingly.**

This is the web crawler that would pull the data from Weather Underground. The website provides csv files for up to a year of weather data. To get 8 years of data, we need to loop through each year individually.

In [6]:
def get_weather(icao):
    base_url = 'https://www.wunderground.com/history/airport/'+icao.upper()
    df = None
    for year in range(2008, 2018):
        url = base_url+'/{0}/10/1/CustomHistory.html'.format(year)
        params = {'dayend': 1, 'monthend': 10, 'yearend': year+1, 'format': 1}
        r = requests.get(url, params=params)
        for line in StringIO(r.text).readlines()[3:]:
            data = line.split(',')[:-1]
            values = {'icao': icao,
                      'date': '{:02}-{:02}-{:02}'.format(*[int(x) for x in data[0].split('-')]), #format dates for SQL
                      'max_temp': data[1],
                      'mean_temp': data[2],
                      'min_temp': data[3],
                      'max_humidity': data[7],
                      'mean_humidity': data[8],
                      'min_humidity': data[9],
                      'precip': data[19],
                      'event': data[21]}
            engine.execute(weather_table.insert(), values)
    print("Pulling weather data over for airport", icao)

Now populate the database for all the airports in the weather table

In [7]:
icao_list = [r[0] for r in engine.execute("SELECT ICAO FROM top_50_airports")]

Now iterate the looping for all 50 airports.

In [8]:
for icao in tqdm(icao_list):
    get_weather(icao)

  2%|▏         | 1/50 [00:00<00:32,  1.51it/s]

Pulling weather data over for airport KATL


  4%|▍         | 2/50 [00:01<00:29,  1.61it/s]

Pulling weather data over for airport KORD


  6%|▌         | 3/50 [00:01<00:28,  1.65it/s]

...

Pulling weather data over for airport KMKE


 98%|█████████▊| 49/50 [00:31<00:00,  1.55it/s]

Pulling weather data over for airport KDAL


100%|██████████| 50/50 [00:32<00:00,  1.55it/s]

Pulling weather data over for airport KIND


**At this moment, we have a database called top_50_airports.db with relevant information and historical weather data of the 50 most travelled airports in the US.**